In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob
# from base.rl.ppo import PPO

In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pthfile = 'ckpt.test.pth'
onnxpath = 'test_conv_pool.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

def crop_map(h, x, crop_size, mode="bilinear"):
    """
    Crops a tensor h centered around location x with size crop_size

    Inputs:
        h - (bs, F, H, W)
        x - (bs, 2) --- (x, y) locations
        crop_size - scalar integer

    Conventions for x:
        The origin is at the top-left, X is rightward, and Y is downward.
    """

    bs, _, H, W = h.size()
    Hby2 = (H - 1) / 2 if H % 2 == 1 else H // 2
    Wby2 = (W - 1) / 2 if W % 2 == 1 else W // 2
    start = -(crop_size - 1) / 2 if crop_size % 2 == 1 else -(crop_size // 2)
    end = start + crop_size - 1
    x_grid = (
        torch.arange(start, end + 1, step=1)
        .unsqueeze(0)
        .expand(crop_size, -1)
        .contiguous()
        .float()
    )
    y_grid = (
        torch.arange(start, end + 1, step=1)
        .unsqueeze(1)
        .expand(-1, crop_size)
        .contiguous()
        .float()
    )
    center_grid = torch.stack([x_grid, y_grid], dim=2).to(
        h.device
    )  # (crop_size, crop_size, 2)

    x_pos = x[:, 0] - Wby2  # (bs, )
    y_pos = x[:, 1] - Hby2  # (bs, )

    crop_grid = center_grid.unsqueeze(0).expand(
        bs, -1, -1, -1
    )  # (bs, crop_size, crop_size, 2)
    crop_grid = crop_grid.contiguous()

    # Convert the grid to (-1, 1) range
    crop_grid[:, :, :, 0] = (
        crop_grid[:, :, :, 0] + x_pos.unsqueeze(1).unsqueeze(2)
    ) / Wby2
    crop_grid[:, :, :, 1] = (
        crop_grid[:, :, :, 1] + y_pos.unsqueeze(1).unsqueeze(2)
    ) / Hby2
    
    print("h.size:", h.size())
    print("crop_grid.size:", crop_grid.size())
    
    h_cropped = F.grid_sample(h, crop_grid, mode=mode)

    return h_cropped

1.4.0
1.10.1
1.9.0


In [64]:
def crop_map(h, crop_size):
    
    bs, ch, H, W = h.size()
    print(h.size())
    
    map_tmp = torch.zeros(1, 4, H+crop_size, W+crop_size)
    map_c1 = torch.ones(H+crop_size, W+crop_size)
    map_tmp[0][0] = map_c1
   
    for i in range(ch):
        map_tmp[0][i][crop_size//2:crop_size//2+H, crop_size//2:crop_size//2+W] = h[0][i]
    
    x_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,0][0]
    y_pos = torch.nonzero(map_tmp[0][3]==1)[:][:,1][0]
    
    # print(map_tmp)
    print("x_pos:{}, y_pos:{}".format(x_pos,y_pos))
    # print(map_tmp.size())
    # print(torch.nonzero(map_tmp[0][3]==1))
    
    output = torch.randn(1, 4, crop_size, crop_size)
    
    for i in range(ch):
        if crop_size%2 == 0:
            output[0][i] = map_tmp[0][i][x_pos-crop_size//2:x_pos+crop_size//2, y_pos-crop_size//2:y_pos+crop_size//2]
        else:
            output[0][i] = map_tmp[0][i][x_pos-crop_size//2:x_pos+crop_size//2+1, y_pos-crop_size//2:y_pos+crop_size//2+1]
            
    return output

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self, G):
        super().__init__()
        self.G = G
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            # nn.BatchNorm2d(1),
            Flatten(),  # (G*G, )
            # nn.Sigmoid(),  # added for non-negative
        )
        
        
    def _get_h12(self, inputs): # inputs needs to be a tensor, i.e., original inputs["map_at_t"], (bs, 4, M, M), channel 3 means one-hot pose, channel 0~1 means global map
        # x = inputs["pose_in_map_at_t"]  # (bs,2)
        # map_at_t (4, m, m)
        # x = torch.nonzero(inputs[0][3]==1)
        h = inputs

        h_1 = crop_map(h, self.G)
        print(h_1.size())
        print(torch.nonzero(h_1[0][3]==1))
        h_2 = F.max_pool2d(h, (self.G, self.G))
# adaptive_max_pool2d
        h_12 = torch.cat([h_1, h_2], dim=1)

        return h_12

    def forward(self, inputs):
        x1 = self._get_h12(inputs)
        x2 = self.actor(x1)
        return x2

In [65]:
actor_net = Global_Actor(240)
actor_net.actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)
    # Initialize model with the pretrained weights

_IncompatibleKeys(missing_keys=['0.weight', '0.bias', '1.weight', '1.bias', '1.running_mean', '1.running_var', '3.weight', '3.bias', '4.weight', '4.bias', '4.running_mean', '4.running_var', '6.weight', '6.bias', '7.weight', '7.bias', '7.running_mean', '7.running_var', '9.weight', '9.bias', '10.weight', '10.bias', '10.running_mean', '10.running_var', '12.weight', '12.bias'], unexpected_keys=['module.0.weight', 'module.0.bias', 'module.1.weight', 'module.1.bias', 'module.1.running_mean', 'module.1.running_var', 'module.1.num_batches_tracked', 'module.3.weight', 'module.3.bias', 'module.4.weight', 'module.4.bias', 'module.4.running_mean', 'module.4.running_var', 'module.4.num_batches_tracked', 'module.6.weight', 'module.6.bias', 'module.7.weight', 'module.7.bias', 'module.7.running_mean', 'module.7.running_var', 'module.7.num_batches_tracked', 'module.9.weight', 'module.9.bias', 'module.10.weight', 'module.10.bias', 'module.10.running_mean', 'module.10.running_var', 'module.10.num_batches

In [66]:
inputs = torch.randn(1, 4, 481, 481)

map3 = torch.zeros(1, 481, 481)
map3[0][20][20] = 1.
print(map3)
print(torch.nonzero(map3==1))
inputs[0][3] = map3
yy = torch.nonzero(inputs[0][3]==1)
print(type(yy))
print(type(yy[:,:2]))
print(yy == yy[:,:2])

x = inputs
print(x.size())

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
tensor([[ 0, 20, 20]])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([[True, True]])
torch.Size([1, 4, 481, 481])


In [67]:
def test():
    M = 481
    model = Global_Actor(240)
    batch_size = 1
    input_shape = (4, M, M)
    
    # Initialize model with the pretrained weights
    map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    loaded_model = torch.load(pthfile, map_location=map_location)
    model.actor.load_state_dict(loaded_model, strict=False)
    
    # set the model to inference mode
    model.eval()
    
    # data type nchw
    # x = torch.rand(batch_size, *input_shape)
    
    torch.onnx.export(model, x, onnxpath)

In [68]:
test()

torch.Size([1, 4, 481, 481])
x_pos:140, y_pos:140
torch.Size([1, 4, 240, 240])
tensor([[120, 120]])


/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  after removing the cwd from sys.path.
/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: TracerWarning: There are 3 live references to the data region being modified when tracing in-place operator copy_ (possibly due to an assignment). This might cause the trace to be incorrect, because all other views that also reference this data will not reflect this change in the trace! On the other hand, if all other views use the same memory chunk, but are disjoint (e.g. are outputs of torch.split), this might still be safe.
  
/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:10: Trace

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 240 and 2 in dimension 2 at /tmp/pip-req-build-ufslq_a9/aten/src/TH/generic/THTensor.cpp:612

In [ ]:
# # Export the model
    torch.onnx.export(model,               # model being run
                    x,                         # model input (or a tuple for multiple inputs)
                    onnxpath,   # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=10,          # the ONNX version to export the model to
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    input_names = ['input'],   # the model's input names
                    output_names = ['output'], # the model's output names
                    dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                  'output' : {0 : 'batch_size'}}
                     )

In [105]:
a = torch.randn((1,2))
print(a.size())
print(a)
b = torch.ones(1,2)
print(b)
print(b.size())

torch.Size([1, 2])
tensor([[0.3203, 0.6363]])
tensor([[1., 1.]])
torch.Size([1, 2])


In [36]:
c = torch.randn(4,6,6)
# print(c)
# print(c[-1])

In [19]:
data = torch.tensor([[1,2,3],[4,2,6]])
target = 4
a = torch.nonzero(data==2)
print(a)
print(a.size())

tensor([[0, 1],
        [1, 1]])
torch.Size([2, 2])


In [153]:
print(*(1,23))
print((1,2))
print(type(*(1,3)))
# a = *(1,2)
print(a)
# print(type(a))

1 23
(1, 2)


TypeError: type() takes 1 or 3 arguments

In [190]:
a = torch.randn(2,3)
print(a)
b = torch.randn(2,3)
print(torch.cat((a,b),dim=1))

tensor([[ 1.6939, -0.3168, -0.5826],
        [ 0.4609,  1.3670, -0.4170]])
tensor([[ 1.6939e+00, -3.1676e-01, -5.8264e-01, -1.2504e+00,  4.0678e-01,
          9.8056e-02],
        [ 4.6094e-01,  1.3670e+00, -4.1705e-01, -1.9146e+00, -1.3156e-03,
          4.9554e-01]])
